In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [34]:
df = pd.read_csv("../../data/tables/X_trainGBM_transposed_with_ID.csv")
df.head()

,ID,ATP7B,ERN1,KRT74,SPECC1L,FOXJ1,OR4K15,PDC,MUC15,C1orf189,...,EIF3F,MAT2A,C2CD4C,M1AP,IQCD,OR2H1,C1QTNF3,RAB42,PTPRJ,GPX7
0,0,49.983944,60.775932,31.239965,46.007948,45.439949,58.503934,40.327955,24.991972,58.503934,...,206.183768,163.015816,29.535967,34.647961,36.919958,31.807964,44.303950,53.391940,45.439949,42.599952
1,0,46.519148,49.501144,34.591161,47.115547,54.272339,58.447134,40.555154,22.066775,48.904745,...,209.336164,171.763007,33.994762,29.223567,33.994762,29.819966,36.380359,53.079540,56.657936,41.151554
2,0,51.213741,42.021531,26.920043,43.991290,55.153259,51.870327,38.082012,25.606870,45.304463,...,225.209141,212.733999,27.576630,30.202975,31.516148,32.172734,46.617636,40.051771,46.617636,45.961049
3,0,45.655397,50.591115,44.421467,33.316100,59.228623,53.058975,50.591115,28.380382,49.357186,...,220.873407,146.837628,22.210734,24.678593,29.614311,25.912523,27.146452,62.930412,38.251819,48.123256
4,0,60.557470,49.547021,39.637617,45.142842,67.163740,56.153291,48.445976,27.526123,37.435527,...,188.278680,140.933749,39.637617,28.627168,33.031347,19.818808,33.031347,42.940752,53.951201,45.142842


In [35]:
ID_col = df.pop('ID')
df.insert(0, 'ID', ID_col)
# df.drop('Unnamed: 0', inplace=True, axis=1)
# df.drop('Unnamed: 0.1', inplace=True, axis=1)
df.to_csv('../../data/tables/X_trainGBM_transposed_with_ID.csv', index=False)
df.head()

,ID,ATP7B,ERN1,KRT74,SPECC1L,FOXJ1,OR4K15,PDC,MUC15,C1orf189,...,EIF3F,MAT2A,C2CD4C,M1AP,IQCD,OR2H1,C1QTNF3,RAB42,PTPRJ,GPX7
0,0,49.983944,60.775932,31.239965,46.007948,45.439949,58.503934,40.327955,24.991972,58.503934,...,206.183768,163.015816,29.535967,34.647961,36.919958,31.807964,44.303950,53.391940,45.439949,42.599952
1,0,46.519148,49.501144,34.591161,47.115547,54.272339,58.447134,40.555154,22.066775,48.904745,...,209.336164,171.763007,33.994762,29.223567,33.994762,29.819966,36.380359,53.079540,56.657936,41.151554
2,0,51.213741,42.021531,26.920043,43.991290,55.153259,51.870327,38.082012,25.606870,45.304463,...,225.209141,212.733999,27.576630,30.202975,31.516148,32.172734,46.617636,40.051771,46.617636,45.961049
3,0,45.655397,50.591115,44.421467,33.316100,59.228623,53.058975,50.591115,28.380382,49.357186,...,220.873407,146.837628,22.210734,24.678593,29.614311,25.912523,27.146452,62.930412,38.251819,48.123256
4,0,60.557470,49.547021,39.637617,45.142842,67.163740,56.153291,48.445976,27.526123,37.435527,...,188.278680,140.933749,39.637617,28.627168,33.031347,19.818808,33.031347,42.940752,53.951201,45.142842


In [36]:
temp_cols=df.columns.tolist()
print(temp_cols)

['ID', 'ATP7B', 'ERN1', 'KRT74', 'SPECC1L', 'FOXJ1', 'OR4K15', 'PDC', 'MUC15', 'C1orf189', 'ZXDC', 'RSAD1', 'ALG1L', 'UBASH3A', 'ZNF274', 'C16orf74', 'ZNF888', 'CXCL17', 'ANAPC10', 'TSPAN31', 'TPST1', 'GABBR2', 'TMEM9', 'RAX', 'FUT5', 'RNFT1', 'TMEM179B', 'MYBPC3', 'SPIN2A', 'INSYN1', 'SLC19A2', 'ZNF333', 'ZNF326', 'TXNDC5', 'RPSA', 'FBXW4', 'PRAMEF18', 'SCGB1D2', 'VPS13D', 'BBS2', 'FBL', 'IL24', 'MFSD2A', 'PRPF18', 'OR2H2', 'BARHL1', 'ANKRD28', 'PPP2R2B', 'SLITRK2', 'TMEM38A', 'CADM4', 'SAA2', 'SERAC1', 'PEDS1', 'COPRS', 'CLGN', 'CDCA4', 'MRPS12', 'ZIC3', 'IFT43', 'OR5D13', 'EPB42', 'SIPA1L2', 'ATP6V0D2', 'NFXL1', 'MRPL30', 'GPRASP2', 'ZBTB1', 'CYP11B2', 'OR2A14', 'FUT8', 'TRIM4', 'FOXRED1', 'IGSF9', 'QTRT1', 'EXTL2', 'LGALSL', 'BPIFB2', 'PCF11', 'KRBOX4', 'PTK6', 'ALDOA', 'PAXX', 'ORAI1', 'CCDC179', 'CST11', 'INTS2', 'TNNC1', 'KCNA3', 'UTP4', 'PIAS2', 'IPO9', 'LACC1', 'ZNF365', 'MAGEA8', 'MT1E', 'PKNOX1', 'NR2C1', 'HBP1', 'BCLAF1', 'CHRNA7', 'SOWAHB', 'SOX15', 'OR10D3', 'C1orf68', 'G

In [37]:
temp_cols.append(temp_cols.pop(temp_cols.index('ID')))
df=df[temp_cols]
df.head()

,ATP7B,ERN1,KRT74,SPECC1L,FOXJ1,OR4K15,PDC,MUC15,C1orf189,ZXDC,...,MAT2A,C2CD4C,M1AP,IQCD,OR2H1,C1QTNF3,RAB42,PTPRJ,GPX7,ID
0,49.983944,60.775932,31.239965,46.007948,45.439949,58.503934,40.327955,24.991972,58.503934,98.263889,...,163.015816,29.535967,34.647961,36.919958,31.807964,44.303950,53.391940,45.439949,42.599952,0
1,46.519148,49.501144,34.591161,47.115547,54.272339,58.447134,40.555154,22.066775,48.904745,99.002289,...,171.763007,33.994762,29.223567,33.994762,29.819966,36.380359,53.079540,56.657936,41.151554,0
2,51.213741,42.021531,26.920043,43.991290,55.153259,51.870327,38.082012,25.606870,45.304463,102.427481,...,212.733999,27.576630,30.202975,31.516148,32.172734,46.617636,40.051771,46.617636,45.961049,0
3,45.655397,50.591115,44.421467,33.316100,59.228623,53.058975,50.591115,28.380382,49.357186,80.205427,...,146.837628,22.210734,24.678593,29.614311,25.912523,27.146452,62.930412,38.251819,48.123256,0
4,60.557470,49.547021,39.637617,45.142842,67.163740,56.153291,48.445976,27.526123,37.435527,89.184638,...,140.933749,39.637617,28.627168,33.031347,19.818808,33.031347,42.940752,53.951201,45.142842,0


In [38]:
df['ID'] = df['ID'].astype('category')
encode_map = {
    '0': 0,
    '1': 1
}

df['ID'].replace(encode_map, inplace=True)

In [39]:

X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
print(X, y)

         ATP7B       ERN1      KRT74    SPECC1L      FOXJ1     OR4K15  \
0    49.983944  60.775932  31.239965  46.007948  45.439949  58.503934   
1    46.519148  49.501144  34.591161  47.115547  54.272339  58.447134   
2    51.213741  42.021531  26.920043  43.991290  55.153259  51.870327   
3    45.655397  50.591115  44.421467  33.316100  59.228623  53.058975   
4    60.557470  49.547021  39.637617  45.142842  67.163740  56.153291   
..         ...        ...        ...        ...        ...        ...   
267  56.538656  69.420882  47.234827  51.528902  52.960260  41.509393   
268  63.872444  59.255159  56.176969  54.637874  44.633756  60.024707   
269  55.789166  62.551489  48.463316  53.535058  56.352692  72.131446   
270  55.107298  52.960260  30.774205  52.960260  20.754697  42.225072   
271  46.889327  48.123256  40.719678  62.313447  27.763417  30.231276   

           PDC      MUC15   C1orf189        ZXDC  ...       EIF3F       MAT2A  \
0    40.327955  24.991972  58.503934   98.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [ ]:
## train data
class TrainData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train),
                       torch.FloatTensor(y_train))
## test data
class TestData(Dataset):

    def __init__(self, X_data):
        self.X_data = X_data

    def __getitem__(self, index):
        return self.X_data[index]

    def __len__ (self):
        return len(self.X_data)


test_data = TestData(torch.FloatTensor(X_test))

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(12, 64)
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)

        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model = BinaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [ ]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()


    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')